# ***Modelo Random Forest Regression***

In [ ]:
%load_ext kedro.ipython 

In [ ]:
catalog.keys()

In [ ]:
catalog.load("model_input_table")

In [ ]:
df_FIFA = catalog.load("model_input_table")

#### **Importaciones**

In [ ]:
# -- Tratamiento de datos --
import numpy as np
import pandas as pd

# -- Gráficos -- 
import seaborn as sns
from matplotlib import style
import matplotlib.pyplot as plt

# -- Procesado y modelado --
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree

# -- Metricas --
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error

# -- GridSearchCV -- 
from sklearn.model_selection import GridSearchCV

# -- CrossValidation -- 
from sklearn.model_selection import cross_val_score

**Conceptos claves**

- Crearemos particiones o nodos en la que se separaran de manera condicional.

- Nodo inicial: llamado es llamado raíz (root)

- Nodos internos: dividen los datos basandose en codiciones > o <. Pueden tener hijos o mas.

- Hojas: Donde no hay mas particiones

RandomForestRegressor: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

Funciones pre-definidas: https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

Funciones personalizadas: https://scikit-learn.org/stable/modules/model_evaluation.html#scoring

In [ ]:
X = df_FIFA[["Reactions","Age"]] 
y = df_FIFA['Value_num']

#### **División de los datos en entrenamiento y prueba**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle = True)

#### **Creación y entrenamiento de Random Forest**

In [ ]:
modelo = RandomForestRegressor(
                                n_estimators=100,       # número de árboles
                                #max_depth=None,         # sin límite de profundidad (puedes probar valores)
                                #min_samples_split=2,    # mínimo de muestras para dividir un nodo
                                #min_samples_leaf=1,     # mínimo de muestras en una hoja
                                random_state=42
                              )
modelo.fit(X_train, y_train)

#### **Predicciones sobre el conjunto de prueba**

In [ ]:
y_pred = modelo.predict(X_test)

#### **Métricas de evaluación**

In [ ]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Métricas de evaluación
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Mostrar resultados
print("-- Métricas de Evaluación del Modelo --")
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Error (MAE):", mae)
print(f"R² (Variance score): {r2:.2f}")

In [ ]:
scores = cross_val_score(modelo, X, y, cv=5, scoring='r2')
print(f"R² promedio CV: {scores.mean():.2f}, desviación estándar: {scores.std():.2f}")


In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(y_test, y_pred, color='skyblue', alpha=0.6)
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--', linewidth=2)  # línea ideal
plt.xlabel('Valores reales')
plt.ylabel('Predicciones')
plt.title('Random Forest Regressor: Predicciones vs Valores reales')
plt.show()


#### **Importancia de las características**

In [ ]:
importances = modelo.feature_importances_
feat_importances = pd.Series(importances, index=X.columns)
feat_importances.sort_values().plot(kind='barh', color='lightgreen')
plt.title("Importancia de las variables")
plt.show()


In [ ]:
# Datos de ejemplo (reemplaza con tus datos)
# X = df[['Overall', 'Age', 'Potential']]
# y = df['Value_num']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Grid completo de hiperparámetros
param_grid_rf = {
    'n_estimators': [10, 20, 50, 100], #     'n_estimators': [100, 200, 300, 500],

    'max_features': ['sqrt', 'log2', None],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 6],
    'criterion': ['squared_error', 'absolute_error'],
    'bootstrap': [True, False]
}

# Crear GridSearchCV
rf = RandomForestRegressor(random_state=42)
grid_search_rf = GridSearchCV(
    estimator=rf,
    param_grid=param_grid_rf,
    cv=5,               # k-fold cross-validation
    scoring='r2',       # optimizar R²
    n_jobs=-1,
    verbose=2
)

# Entrenar
grid_search_rf.fit(X_train, y_train)

# Mejor modelo
best_rf = grid_search_rf.best_estimator_
print("Mejores hiperparámetros:", grid_search_rf.best_params_)

# Predicciones
y_pred = best_rf.predict(X_test)

# Evaluación
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\n Evaluación del mejor modelo:")
print("MSE: ", mse)
print("MAE: ",  mae)
print("RMSE: ", rmse)
print("R^2: ", r2)

print(df_metrics)